<a href="https://colab.research.google.com/github/phice421/Redes-Neurais/blob/main/Redes_Neurais_Artificiais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IA & Big Data**



---
**Redes Neurais**

---

In [ ]:
# Importando as bibliotecas necessarias para o projeto
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils import plot_model

# Ajuste nos dados
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Métricas de Desempenho
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report

# Plot dos gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Google Drive
from google_drive_downloader import GoogleDriveDownloader as gdd


## Conjunto de dados da aula

O robô **SCITOS G5** foi objeto de estudo de um modelo de um robô que segue parede de forma autonoma. Nesse estudo foram coletados dados dos 24 sensores ultrassônicos instalados ao redor do robô (ver imagem 1).


**Referencia do dataset e do artigo dessa questão:**

Ananda L. Freire, Guilherme A. Barreto, Marcus Veloso and Antonio T. Varela (2009),
	"Short-Term Memory Mechanisms in Neural Network Learning of Robot Navigation
	Tasks: A Case Study". Proceedings of the 6th Latin American Robotics Symposium (LARS'2009),
	Valparaíso-Chile, pages 1-6, DOI: 10.1109/LARS.2009.5418323

<img src='https://drive.google.com/uc?id=1Sd3MvhItVs-4ufbmtcMBsWkdLZJnClud' width=300>


*Figura 1 - robô SCITOS G5*




O objetivo do robô era conseguir seguir uma parede evitando eventuais colisões. Esse robô foi testado com diversos algoritmos e os dados dos valores desses sensores, assim como o movimento que o robô deveria fazer, foram armazenados em um conjunto de dados. A imagem a seguir ilustra as rotas realizadas pelo robô.


<img src="https://drive.google.com/uc?id=1xey3l885AKnBiB9cR_3S9SglY9oLd0Qu" width=800>

*Figura 2 - Exemplo de trajetória executada pelo robô*

A seguir temos uma descrição dos diversos sensores do robô instalados no robô

  
1. US1: Sensor ultrassônico na parte frontal do robô (ângulo de referência: 180°) - (numeric: real)

2. US2: Leitura do sensor ultrassônico (ângulo de referência: -165°) - (numeric: real)

3. US3: Leitura do sensor ultrassônico (ângulo de referência: -150°) - (numeric: real)

4. US4: Leitura do sensor ultrassônico (ângulo de referência: -135°) - (numeric: real)

5. US5: Leitura do sensor ultrassônico (ângulo de referência: -120°) - (numeric: real)

6. US6: Leitura do sensor ultrassônico (ângulo de referência: -105°) - (numeric: real)

7. US7: Leitura do sensor ultrassônico (ângulo de referência: -90°) - (numeric: real)

8. US8: Leitura do sensor ultrassônico (ângulo de referência: -75°) - (numeric: real)

9. US9: Leitura do sensor ultrassônico (ângulo de referência: -60°) - (numeric: real)

10. US10: Leitura do sensor ultrassônico (ângulo de referência: -45°) - (numeric: real)

11. US11: Leitura do sensor ultrassônico (ângulo de referência: -30°) - (numeric: real)

12. US12: Leitura do sensor ultrassônico (ângulo de referência: -15°) - (numeric: real)

13. US13: Sensor ultrassônico na parte traseira do robô  t (ângulo de referência: 0°) - (numeric: real)

14. US14: Leitura do sensor ultrassônico (ângulo de referência: 15°) - (numeric: real)

15. US15: Leitura do sensor ultrassônico (ângulo de referência: 30°) - (numeric: real)

16. US16: Leitura do sensor ultrassônico (ângulo de referência: 45°) - (numeric: real)

17. US17: Leitura do sensor ultrassônico (ângulo de referência: 60°) - (numeric: real)

18. US18: Leitura do sensor ultrassônico (ângulo de referência: 75°) - (numeric: real)

19. US19: Leitura do sensor ultrassônico (ângulo de referência: 90°) - (numeric: real)

20. US20: Leitura do sensor ultrassônico (ângulo de referência: 105°) - (numeric: real)

21. US21: Leitura do sensor ultrassônico (ângulo de referência: 120°) - (numeric: real)

22. US22: Leitura do sensor ultrassônico (ângulo de referência: 135°) - (numeric: real)

23. US23: Leitura do sensor ultrassônico (ângulo de referência: 150°) - (numeric: real)

24. US24: Leitura do sensor ultrassônico (ângulo de referência: 165°) - (numeric: real)

25. Classes (target):

    -- Move-Forward - Mover para frente

    -- Slight-Right-Turn - Curva leve à direita

    -- Sharp-Right-Turn - Curva acentuada à direita

    -- Slight-Left-Turn - Curva leve à direita


## Tarefa #1: Recebendo os dados

In [ ]:
data_google_id = '18jiZdkIufjJIbeWxk3URWrIQHO0IPWxZ'
gdd.download_file_from_google_drive(file_id=data_google_id,
                                    dest_path = './dados.csv',
                                    showsize = True,
                                    overwrite=True)
dados = pd.read_csv("dados.csv", sep=',')

## Tarefa #2: Analise Exploratória dos Dados

In [ ]:
dados.head()

In [ ]:
dados.info()

Desse conjunto de dados podemos ver que o mesmo possui váriaveis do float. A única coluna do tipo object é a coluna com a ação que o robô deve executar. Logo não há a necessidade do uso de *One Hot Encoding* ou *Label Encoding*


In [ ]:
dados.shape

In [ ]:
dados.isnull().sum()

Temos valores nulos que deverão ser tratados! Iremos substituir pela mediana de cada coluna.

In [ ]:
dados.loc[:,'sensor_1':'sensor_13'].describe()

In [ ]:
dados.loc[:,'sensor_13':].describe()

Todas as colunas com os sensores tem valores da mesma ordem de grandeza. Logo não há a necessidade de alterar a escala dos dados.

Vamos analisar as possibilidades de movimentos do robô:

In [ ]:
sns.countplot(data=dados, x='funcao_robo')

In [ ]:
dados['funcao_robo'].value_counts()

## Tarefa #3: Corrigindo o problema dos dados

### Substituindo os valores nulos pela mediana

Para as colunas dos valores dos sensores iremos substituir os mesmos pela mediana de cada coluna:

In [ ]:
colunas = list(dados.columns)
colunas

In [ ]:
colunas.pop()
colunas

In [ ]:
for col in colunas:
  mediana = dados[col].median()
  dados[col].fillna(mediana, inplace = True)

In [ ]:
dados.isnull().sum()

Na coluna função_robô temos apenas 14 valores nulos. Iremos remover esses valores pois temos mais de 5000 linhas nesse conjunto de dados.

In [ ]:
dados.dropna(inplace=True)

In [ ]:
dados.isnull().sum()

### Ajustando o desbalanceamento da coluna de saída dos dados (EXTRA - Datasets desbalanceados)

In [ ]:
# Dividindo os dados em entrada e saída:
X = dados.drop(columns=["funcao_robo"])
# Criando uma variavel para os labels
y = dados["funcao_robo"]

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
rus = RandomOverSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X,y)

In [ ]:
dados = pd.concat([X_resampled, y_resampled], axis = 1)

In [ ]:
sns.countplot(data=dados, x='funcao_robo')

### Ajustando os dados para a Rede Neural

No nosso modelo iremos ter quatro saídas na nossa rede. Cada uma delas com a probabilidade de nossos dados pertencerem a classe 0, 1, 2 e 3 (4 possíveis movimentos do robô). Para isso, precisamos transformar nossos dados para que possamos ter as quatro saídas nos nossos dados. Como iremos usar o `Keras` e não o `Tensorflow` devemos adaptar nossos dados para esse novo cenário:

In [ ]:
X = dados.drop(columns='funcao_robo')
y = dados['funcao_robo']
y.unique()

In [ ]:
dicionario_subs = {}

for i, rotulo in enumerate(y.unique()):
  dicionario_subs[rotulo] = i

In [ ]:
dicionario_subs

In [ ]:
y.replace(dicionario_subs, inplace = True)

In [ ]:
y.unique()

In [ ]:
y.shape

In [ ]:
y_keras = tf.keras.utils.to_categorical(y)
y_keras.shape

In [ ]:
y_keras

In [ ]:
y

##Tarefa #4: Redes Neurais Artificias para Classificação de Dados:

### Tarefa #A: Dividindo o conjunto de dados (Entrada/Saída e Treinamento/Teste)

O procedimento com o conjunto de dados será o mesmo, primeiro iremos dividir os mesmos em conjunto de treinamento e de teste.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y_keras,
                                                    test_size=0.2,
                                                    random_state=42)

### Tarefa #B: Criando a Rede Neural com o Keras

Vamos definir um modelo com as seguintes camadas:

*   Entrada com 24 valores (24 atributos dos nossos dados)
*   Primeira camada com 8 neurônios e função de ativação por ReLU
*   Camada Oculta com 8 neurônios e função de ativação por ReLU
*   Saída da rede com 4 neurônios e função de ativação por softmax


In [ ]:
# Criando o modelo para fazer a classificação dos dados:
model = Sequential()

# Camada de Entrada:
model.add(Input(shape=24, # 24 sensores
                name = "Entrada"))
# Primeira Camada:
model.add(Dense(8,
                use_bias=True, # Use o bias
                bias_initializer = 'zeros', # inicializando bias em zeros
                activation='relu', # função de ativação ReLU
                name = 'Primeira_Camada'))
# Camada Oculta:
model.add(Dense(8,
                use_bias=True, # Use o bias
                bias_initializer = 'zeros', # inicializando bias em zeros
                activation='relu', # função de ativação ReLU
                name = 'Camada_Oculta'))
# Camada de Saída:
model.add(Dense(4,
                use_bias=True, # Use o bias
                bias_initializer = 'zeros', # inicializando bias em zeros
                activation='softmax', # função de ativação softmax
                name = 'Camada_Saida'))

In [ ]:
model.summary()

In [ ]:
plot_model(model,
           to_file = 'model_plot.png',
           show_shapes = True,
           show_layer_names=True,
           show_layer_activations=True)

### Tarefa #C: Treinando a Rede Neural com o Keras

Vamos fazer o treinamento do nosso modelo usando o categorical crossentropy como função de custo, SGD como algoritmo de otimização dos parâmetros, taxa de aprendizado de 0,001 e métrica no treinamento com a acurácia.

Realizando o treinamento com 1500 épocas, com batch_size de 10 e 20% dos dados utilizados para a validação do modelo:

Vamos plotar um gráfico para analisar o comportamento da função custo durante o treinamento:

Podemos ver que o valor da função custo manteve o mesmo padrão para o conjunto de treinamento e de validação, o que indica que não houver overfitting no nosso treinamento

Salvando o modelo que foi criado:

### Tarefa #D: Métricas de Desempenho

Vamos checar o valor de saída da rede para o conjunto de teste:

O padrão obtido seria a porcentagem de cada um dos valores do conjunto de teste pertencerem a cada uma das classes (0, 1 e 2). Aquele com o maior valor, é a classe com a qual o nosso modelo fez a previsão. Vamos agora transforma isso em um array com os valores das classes e não a probabilidade de cada uma delas.

Vamos agora checar a matriz de confusão dos nossos dados:

Por fim, vamos obter as nossas métricas de precisão, revocação e acurácia: